In [1]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.optimizers import RMSprop  

from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop


import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 12, 5

Using TensorFlow backend.


In [2]:
path = 'databook.txt' ## We will use Friedrich Nietzsche’s Beyond Good and Evil as a training corpus for our model.
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600894


In [5]:
list_str = text.split()
words = sorted(set(list_str) ## list of distinct words in the corpus

word_indices = dict((c, i) for i, c in enumerate(chars)) ## char to index maps
indices_word = dict((i, c) for i, c in enumerate(chars)) ## index to char maps

print(f'unique chars: {len(chars)}')

unique chars: 99111


In [10]:
SEQUENCE_LENGTH = 10
step = 1
sentences = []
next_words = []
for i in range(0, len(list_str) - SEQUENCE_LENGTH):
    sentences.append(list_str[i: i + SEQUENCE_LENGTH])
    next_words.append(list_str[i + SEQUENCE_LENGTH])
print(f'num training examples: {len(sentences)}')

num training examples: 99101


In [11]:
for i in range(0,5):
  print(sentences[i])
  print('--'*20)
  print(next_words[i])
  print('='*30)

['preface', 'supposing', 'that', 'truth', 'is', 'a', 'woman--what', 'then?', 'is', 'there']
----------------------------------------
not
['supposing', 'that', 'truth', 'is', 'a', 'woman--what', 'then?', 'is', 'there', 'not']
----------------------------------------
ground
['that', 'truth', 'is', 'a', 'woman--what', 'then?', 'is', 'there', 'not', 'ground']
----------------------------------------
for
['truth', 'is', 'a', 'woman--what', 'then?', 'is', 'there', 'not', 'ground', 'for']
----------------------------------------
suspecting
['is', 'a', 'woman--what', 'then?', 'is', 'there', 'not', 'ground', 'for', 'suspecting']
----------------------------------------
that


In [0]:
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)), dtype=np.bool) ## 99101 X 10 X 57 arrays of zeros
y = np.zeros((len(sentences), len(chars)), dtype=np.bool) ## 200285 X 57 arrays of zeros
for i, sentence in enumerate(sentences):  ## All the 200285 are indexed
    for t, char in enumerate(sentence): ## In every sentence, each character is indexed
        X[i, t, char_indices[char]] = 1  ## every char in each sentence is marked as 1---- 3D one hot vector
    y[i, char_indices[next_chars[i]]] = 1 ## in very sentence next char to come is marked as 1---- 2D one hot vector